In [12]:
from requests import get
from bs4 import BeautifulSoup
import urllib.request as urllib
import re

In [13]:
def clean_answer(input_text):
    """function that cleans the answer texts"""  
    answer = input_text.strip('\n')
    answer = answer.replace('\n',' ')
    return answer  

In [14]:
def clean_links(link_list):
    '''function that cleans links depending on their target location'''
    clean_links = []
    
    for link in link_list:
        if link == None: # no link in answer
            clean_links.append(None)
            pass
        
        else:
            link = link['href']
            if link[0] == '?': # target is sub page of the faq
                f_link = 'https://www.rug.nl/education/faq/' + link
                clean_links.append(f_link)
            
            elif link[0] == '/': # target is sub page of the rug site
                f_link = 'https://www.rug.nl' + link
                #print(f_link)
                clean_links.append(f_link)
            
            else: # target is a different link
                clean_links.append(link)
                #print(link)

    return clean_links

In [158]:
def retrieve_data(question_list, answer_list, link_list, url, heading_list):
    """function retrieves questions and answers from the specific RUG pages"""
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    data = html_soup.findAll('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
        
    for links in data:
        link = links.findAll('a')
        
        for a in link:
            if a['href'][0] == "?": #check if link is an internal one
                current_url = 'https://www.rug.nl/education/faq/'+a['href']
                response = get(current_url)
                #print('1',current_url)
                html_soup = BeautifulSoup(response.text, 'html.parser')
                questions = html_soup.find_all(['h2'])
                content = html_soup.findAll('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                s_content = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                
"""Dit stuk heb ik toegeveogd en dat leest nu de antwoorden uit de sublinkjes uit maar nu
    ziet ie dus niet meer de antwoorden die geen linkjes hebben zoals 
    Welke opleidingen geven rechtstreeks toelating tot de universiteit?
    Ik denk dat dit stukje in een van de if statements hieronder moet maar ik weet nog even niet hoe"""
                for sublink in content:
                    linklist = sublink.findAll('ul')
                    linkinanswer =sublink.findAll('ul', {'class': 'rug-list--bullets rug-mv-xs'})
                    for linkonpage in linklist:
                        if linkonpage not in linkinanswer:
                            nextpage = linkonpage.findAll('a')
                            for pagelink in nextpage:
                                questionlink = pagelink.text
                                #print(questionlink)
                                if pagelink['href'][0] == "?":
                                    current_url = 'https://www.rug.nl/education/faq/'+pagelink['href']
                                else: 
                                    current_url = 'https://www.rug.nl'+ pagelink['href']
                                response = get(current_url)
                                html_soup = BeautifulSoup(response.text, 'html.parser')
                                ans = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                                #print(ans)
                                #catch errors when external page is formatted differently
                                try:
                                    answer = clean_answer(ans.text)
                                    ans_link = ans.a
                                    link_list.append(ans_link)
                                    answer_list.append(answer)
                                    question_list.append(questionlink)
                                
                                #Structure of external url is different
                                except(AttributeError, KeyError) as error: 
                                    ans = html_soup.find(['p'])
                                    ans_link = ans.a
                                    link_list.append(ans_link)
                                    answer = clean_answer(ans.text)
                                    answer_list.append(answer)
                                    question_list.append(questionlink)

                            
                #tot hierboven
                
                if len(content) == len(questions): #check structure of current questions and answers
                    for question, answer in zip(questions, content):      
                        if question.text in heading_list: #check if question contains sub questions
                            links = answer.find_all('a')

                            #access the subquestions
                            for a in links:
                                q = a.text
                                
                                if a['href'][0] == "?":
                                    current_url = 'https://www.rug.nl/education/faq/'+a['href']
                                else: 
                                    current_url = a['href']

                                response = get(current_url)
                                html_soup = BeautifulSoup(response.text, 'html.parser')
                                ans = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})

                                #catch errors when external page is formatted differently
                                try:
                                    answer = clean_answer(ans.text)
                                    ans_link = ans.a
                                    link_list.append(ans_link)
                                    answer_list.append(answer)
                                    question_list.append(q)
                                
                                #Structure of external url is different
                                except(AttributeError, KeyError) as error: 
                                    ans = html_soup.find(['p'])
                                    ans_link = ans.a
                                    link_list.append(ans_link)
                                    answer = clean_answer(ans.text)
                                    answer_list.append(answer)
                                    question_list.append(q)
                        else:      
                            question_list.append(question.text)
                            ans_link = answer.a
                            link_list.append(ans_link)
                            answer = clean_answer(answer.text)
                            answer_list.append(answer)   
                            
                else:
                    question = html_soup.find(['h1'])
                    question_list.append(question.text)
                    ans_link = s_content.a
                    link_list.append(ans_link)
                    answer = clean_answer(s_content.text)
                    answer_list.append(answer)
                    
            else:
                question = a.text
                ans_link = links.a
                link_list.append(ans_link)
                answer = clean_answer(links.text)
                question_list.append(question)
                answer_list.append(answer)
        
    return question_list, answer_list, link_list


In [162]:
def write_aiml_file(question_list, answer_list, clean_links):
    '''function that automatically generates aiml file from questions, answers and links in answers'''
    
    with open('aiml_base_kar2.aiml', 'w') as f:
        for vraag, antwoord, link in zip(question_list, answer_list, clean_links):

            if link is not None: # there is a link in the answer
                f.write('<category>'+'\n'+ '<pattern>'+ '\n'+ vraag+ '\n'+ '</pattern>'+'\n'+
                '<template>'+ '\n'+ antwoord + '\n'+ '<button>' + '\n' + '<text>' + '\n'+ 'Klik hier' 
                + '\n' + '</text>' + '\n' + '<url>'+ '\n' +link+ '\n' + '</url>' + '\n' + '</button>' + '\n' '</template>'+'\n'+'</category>'+'\n'+'\n')

            else:
                f.write('<category>'+'\n'+
              '<pattern>'+ '\n'+ vraag+ '\n'+ '</pattern>'+'\n'+
              '<template>'+ '\n'+ antwoord+ '\n'+ '</template>'+'\n'+
              '</category>'+'\n')
    f.close()

In [17]:
url_list = ['https://www.rug.nl/education/faq/?tcid=verint_3_7394_7394',
            'https://www.rug.nl/education/faq/?tcid=verint_3_7395_7395']

heading_list = []

for url in url_list:
    html_page = urllib.urlopen(url)
    soup = BeautifulSoup(html_page,'html.parser')
    for h in soup.findAll('h2'):
        heading_list.append(h.text)

print(heading_list)

['Aanmelding en toelating', 'Inschrijven', 'Herinschrijven', 'Uitschrijven', 'Studielink', 'Meer informatie ...', 'Collegegeld', 'Financiële ondersteuning', 'Profileringsfonds', 'Studiefinanciering (DUO)', 'Studiebeurzen', 'Studiekosten']


In [159]:
question_list = []
answer_list = []
link_list = []

for url in url_list:
    question_list, answer_list, link_list = retrieve_data(question_list, answer_list, link_list, url, heading_list)


In [160]:
for vraag in question_list:
    question_list[question_list.index(vraag)] = vraag.replace('?', '')    

In [161]:
for antwoord in answer_list:
    if re.search('Stel je vraag', antwoord):
        answer_list[answer_list.index(antwoord)] = re.sub('\Stel je vraag$', '', antwoord)
    else:
        pass


In [163]:
clean_links = clean_links(link_list)
write_aiml_file(question_list, answer_list, clean_links)

In [164]:
for vraag, antwoord in zip(question_list, answer_list):
    print(vraag,antwoord,'\n')

Wat is het verschil tussen aanmelden en inschrijven Aanmelden en inschrijven zijn twee verschillende acties. Je meldt je via Studielink aan voor een opleiding. Je wordt pas ingeschreven als je alle stappen van de inschrijfprocedure hebt doorlopen.  Informatie over de inschrijfprocedures voor Nederlandse studenten vind je op de onderstaande pagina's op onze website:   Bachelor aanmelding en inschrijving   Master aanmelding en inschrijving       

Hoe meld ik me aan voor een bacheloropleiding Informatie over de inschrijfprocedure voor Nederlandse studenten vind je op de pagina Inschrijfprocedure bacheloropleiding op onze website.  Informatie over de inschrijfprocedure voor internationale studenten en Nederlandse studenten met een buitenlandse vooropleiding vind je op de pagina Admissions guide op onze website.     

Hoe meld ik me aan voor een masteropleiding Informatie over de inschrijfprocedure voor Nederlandse studenten vind je op de pagina Inschrijfprocedure masteropleiding op onze w

In [ ]:
#Gijs 
#add headings which contain sub_links to this list
heading_list = ["Aanmelding- en selectieprocedure","Onderwijs - honoursprogramma","Bindend Studieadvies (BSA)", 
                "Studeren met een functiebeperking", "Studiedips en andere studieproblemen.", "Studiekeuze", 
                "Studievertraging", "Honours College", "Academische ontwikkeling", "Titulatuur / graden", "Alumni"] 

#currently contains 3 start pages: naast je studie, begeleiding and studieloopbaan 
url_list = ['https://www.rug.nl/education/faq/?tcid=verint_3_7412_7412', 'https://www.rug.nl/education/faq/?tcid=verint_3_7398_7398', 
            'https://www.rug.nl/education/faq/?tcid=verint_3_7399_7399']


In [6]:
# Iris

url_list = ['https://www.rug.nl/education/faq/?tcid=verint_3_7400_7400',
            'https://www.rug.nl/education/faq/?tcid=verint_3_7402_7402', 
            'https://www.rug.nl/education/faq/?tcid=verint_3_7401_7401']

heading_list =['Adressen en openingstijden', 'Voorzieningen', 'Formulieren (overzicht)', 
                'Aanvragen verklaringen (student)', 'Regelingen (overzicht)', 'Handleidingen (overzicht)']

In [157]:
#hiermee heb ik die linkjes gevonden

response = get('https://www.rug.nl/education/faq/?tcid=verint_3_7394_7394')
html_soup = BeautifulSoup(response.text, 'html.parser')
data = html_soup.findAll('div',{'class':'rug-clearfix rug-theme--content rug-mb'})

for links in data:
    link = links.findAll('a')
    for a in link:
        if a['href'][0] == "?": #check if link is an internal one
            current_url = 'https://www.rug.nl/education/faq/'+a['href']
            response = get(current_url)

            html_soup = BeautifulSoup(response.text, 'html.parser')
            data = html_soup.findAll('div',{'class':'rug-clearfix rug-theme--content rug-mb'})

            for links in data:
                link = links.findAll('ul')
                linkinanswer =links.findAll('ul', {'class': 'rug-list--bullets rug-mv-xs'})
                for p in link:
                    if p not in linkinanswer:
                        nextpage = p.findAll('a')
                        for i in nextpage:
                            print(i.text)
                            if i['href'][0] == "?":
                                current_url = 'https://www.rug.nl/education/faq/'+i['href']
                            else: 
                                current_url = 'https://www.rug.nl'+i['href']
                            print('3', current_url)
                            response = get(current_url)
                            



#url = 'https://www.rug.nl/education/faq/?tcid=verint_3_7394_7844'
#url = 'https://www.rug.nl/education/faq/?tcid=verint_3_7394_7843'





Ben ik met een vwo-diploma toelaatbaar tot een bacheloropleiding?
3 https://www.rug.nl/education/faq/?tcid=verint_3_7394_10541
Ben ik met een vwo-diploma oude stijl toelaatbaar tot een bacheloropleiding?
3 https://www.rug.nl/education/faq/?tcid=verint_3_7394_10542
Wat kan ik doen als mijn profiel niet toereikend is voor de opleiding van mijn voorkeur?
3 https://www.rug.nl/education/faq/?tcid=verint_3_7394_10543
Kan ik ook toegelaten worden met deelcertificaten als ik mijn VWO diploma niet haal?
3 https://www.rug.nl/education/faq/?tcid=verint_3_7394_10544
Ben ik met een hbo-propedeuse of -diploma toelaatbaar tot een universitaire bacheloropleiding?
3 https://www.rug.nl/education/faq/?tcid=verint_3_7394_10545
Maakt het uit welke hbo-propedeuse ik heb gehaald als ik naar de universiteit wil?
3 https://www.rug.nl/education/faq/?tcid=verint_3_7394_10546
Kan ik instromen in een hoger studiejaar?
3 https://www.rug.nl/education/faq/?tcid=verint_3_7394_10547
Kan ik met een hbo-bachelordiploma i

In [11]:
#inspect questions and answers
for vraag, antwoord in zip(question_list, answer_list):
    print(vraag,antwoord,'\n')

Onderwijs-
en examenregelingen per faculteit In de Onderwijs- en Examenregeling (OER) zijn de rechten en plichten van een student over allerlei zaken vastgelegd, zoals opbouw van de opleiding, toelatingseisen tot een bepaald jaar van de opleiding, de volgorde van studieonderdelen, de practica, de vorm van tentamens en studiebegeleiding.  Een overzicht van de Onderwijs- en Examenregelingen vind je op de pagina Onderwijs- en examenregelingen per faculteit op onze website. 

Studentenstatuut In het studentenstatuut zijn de rechten en plichten van zowel studenten als de Rijksuniversiteit Groningen vastgelegd.  Meer informatie vind je op de pagina Studentenstatuut op onze website. 

Centraal Loket Rechtsbescherming
Studenten Je kunt een klacht, bezwaar of beroep indienen bij het Centraal Loket Rechtsbescherming Studenten (CLRS). 

Vertrouwenspersoon Er zijn op de universiteit twee instanties waar je terecht kunt met een vertrouwelijke vraag, een klacht of een bezwaar/beroepsprocedure.    Ve

In [ ]:
#inspect questions and answers
for vraag, antwoord in zip(question_list, answer_list):
    print(vraag,antwoord,'\n')

In [ ]:
#inspect questions and answers
for vraag, antwoord in zip(question_list, answer_list):
    print('<category>'+'\n'+
          '<pattern>'+ '\n'+ vraag+ '\n'+ '</pattern>'+'\n'+
          '<template>'+ '\n'+ antwoord+ '\n'+ '</template>'+'\n'+
          '</category>'+'\n')
    #print(aiml)

#total_aiml ='<?xml version="1.0" encoding="UTF-8"?>', '\nl','<aiml version="2.0">', '\n', aiml, '\n','</aiml>'
#print(total_aiml)
    

In [ ]:
html_page = urllib.urlopen("https://www.rug.nl/education/faq/")
soup = BeautifulSoup(html_page,'html.parser')
#print(soup.prettify())
                       
#link = soup.findAll("a", {"class": re.compile("rug-link--caret")})

#https://www.rug.nl/education/faq/?tcid=verint_3_7394_7526
new_url = []

#soup = BeautifulSoup(html_page)
for link in soup.findAll('a', {'class': "rug-link--caret"}):
    if len(link.get('class')) == 1:
        new_url.append(link.get('href'))

new_url

In [ ]:
address = "https://www.rug.nl/education/faq/" + link_vraag[0]
html_page = urllib.urlopen(address)
soup = BeautifulSoup(html_page,'html.parser')


for answer in soup.findAll('div', {'class':"rug-clearfix rug-theme--content rug-mb"}):
    if re.search('vraag', answer.text):
        ans = answer.text.replace('Stel je vraag', '')
        print(ans)
        antwoord.append(ans)
        
        
print(antwoord)
        

            

    

In [ ]:
# Deze print alle vragen en antwoorden van de websites in new_url. Ik moet nog losse lijsten maken dus dat
# je per nieuwe url (subonderwerp) een lijst met vragen en een lijst met antwoorden hebt. Die lijsten voeg je weer
# aan 

html = []

vragen_all = []
link_vraag_all = []
antwoord_all = []

i=0
for l in new_url:
    address = "https://www.rug.nl/education/faq/" + l
    print("Questions from:", address)
    html_page = urllib.urlopen(address)
    soup = BeautifulSoup(html_page,'html.parser')
    
    vragen = []
    link_vraag = []
    antwoord = []

    for vraaglijst in soup.findAll('div', {'class': "rug-clearfix rug-theme--content rug-mb"}):
        html.append(vraaglijst)


    for vraag in html:
        for vraaglink in vraag.findAll('a'):
            vlink = vraaglink.get('href')
            vtext = vraaglink.text
            link_vraag.append(vlink)
            vragen.append(vtext)

    for link in link_vraag:
        #print(vragen[link_vraag.index(link)])
        #print(i)
        i+=1
        address = "https://www.rug.nl/education/faq/" + link
        #print(address)
        html_page = urllib.urlopen(address)
        soup = BeautifulSoup(html_page,'html.parser')


        for answer in soup.findAll('div', {'class':"rug-clearfix rug-theme--content rug-mb"}):
            if re.search('vraag', answer.text) == None:
                #print(answer.text)
                antwoord.append(answer.text)
                
    vragen_all.append(vragen)
    link_vraag_all.append(link_vraag)
    antwoord_all.append(antwoord)

    
len(vragen_all)
len(link_vraag_all)
len(antwoord_all)

In [ ]:
vragen = []
link_vraag = []
antwoord = []

html = []



address = "https://www.rug.nl/education/faq/" + new_url[0]
html_page = urllib.urlopen(address)
soup = BeautifulSoup(html_page,'html.parser')

for vraaglijst in soup.findAll('div', {'class': "rug-clearfix rug-theme--content rug-mb"}):
    html.append(vraaglijst)

    
for vraag in html:
    for vraaglink in vraag.findAll('a'):
        vlink = vraaglink.get('href')
        vtext = vraaglink.text
        link_vraag.append(vlink)
        vragen.append(vtext)

print(vragen)

    

In [ ]:
## get question links of page

address = "https://www.rug.nl/education/faq/" + new_url[0]
html_page = urllib.urlopen(address)
soup = BeautifulSoup(html_page,'html.parser')

vraag = []
link_vraag = []
antwoord = []

for link in soup.find('div', {'class': "rug-clearfix rug-theme--content rug-mb"}).findAll('a'):
    vraag.append(link.text)
    link_vraag.append(link.get('href'))
    
print(vraag)
print(link_vraag)

In [ ]:
def retrieve_data(question_list, answer_list,url,heading_list):
    """function retrieves questions and answers from the specific RUG pages"""
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    data = html_soup.findAll('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
    
    for links in data:
        link = links.findAll('a')
        
        for a in link:
            if a['href'][0] == "?": #check if link is an internal one
                current_url = 'https://www.rug.nl/education/faq/'+a['href']
                response = get(current_url)
                html_soup = BeautifulSoup(response.text, 'html.parser')
                questions = html_soup.find_all(['h2'])
                content = html_soup.findAll('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                s_content = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                
                if len(content) == len(questions): #check structure of current questions and answers
                    for question, answer in zip(questions, content):      
                        if question.text in heading_list: #check if question contains sub questions
                            links = answer.find_all('a')

                            #access the subquestions
                            for a in links:
                                q = a.text
                                
                                if a['href'][0] == "?":
                                    current_url = 'https://www.rug.nl/education/faq/'+a['href']
                                else: 
                                    current_url = a['href']

                                response = get(current_url)
                                html_soup = BeautifulSoup(response.text, 'html.parser')
                                ans = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})

                                #catch errors when external page is formatted differently
                                try:
                                    if re.search('Stel je vraag', ans.text) == True:
                                        print('check1')
                                        answer = clean_answer(ans.text)
                                        answer = answer.replace('Stel je vraag', '')
                                        answer_list.append(answer)
                                        question_list.append(q)
                                
                                #Structure of external url is different
                                except(AttributeError, KeyError) as error: 
                                    ans = html_soup.find(['p'])
                                    if re.search('Stel je vraag', ans.text) == True:
                                        print('check2')
                                        answer = clean_answer(ans.text)
                                        answer = answer.replace('Stel je vraag', '')
                                        answer_list.append(answer)
                                        question_list.append(q)
                        else:
                            if re.search('Stel je vraag', answer.text) == True:
                                print('check3')
                                question_list.append(question.text)
                                answer = clean_answer(answer.text)
                                answer = answer.replace('Stel je vraag', '')
                                answer_list.append(answer)   
                            
                else:
                    if re.search('Stel je vraag', s_content.text) == True:
                        print('check4')
                        question = html_soup.find(['h1'])
                        question_list.append(question.text)
                        answer = clean_answer(s_content.text)
                        answer = answer.replace('Stel je vraag', '')
                        answer_list.append(answer)
                    
            else:
                if re.search('Stel je vraag', links.text) == True:
                    print('check5')
                    question = a.text
                    answer = clean_answer(links.text)
                    answer = answer.replace('Stel je vraag', '')
                    question_list.append(question)
                    answer_list.append(answer)
        
    return question_list, answer_list


In [ ]:
print(len(vraag))